In [ ]:
# 目的
このノートブックは、Titanic（タイタニック号の乗客データ）を使って、
機械学習用に使いやすくするための「前処理」と「データを理解するための簡単なグラフ」を作る。

# データ概要
- データ: train.csv (891行 × 12列)
- 目的変数: Survived (生存: 0=死亡, 1=生存)
- 主な特徴量:
  - Pclass: 客室クラス (1〜3)
  - Sex: 性別
  - Age: 年齢
  - SibSp: 同乗した兄弟/配偶者の数
  - Parch: 同乗した親/子供の数
  - Fare: 運賃
  - Embarked: 乗船した港 (S, C, Q)
- 前処理済みデータ: cleaned_titanic.csv (891行 × 11列)

- Ageは中央値で補完：外れ値耐性のため
- Embarkedは最頻値：欠損2件で全体傾向を崩さない
- Cabinは有無フラグ化：欠損多数で詳細は不採用
- Sexは2値化、Embarkedはone-hot：順序誤解を防ぐ

# 出力（後で再利用できる形）
- 前処理後のCSV: cleaned_titanic.csv
- 参考図: figs/age_hist.png, figs/sex_count.png, figs/corr_numeric.png

In [ ]:
# 1. データ読込
# 2. データの状態確認（欠損・型・ざっくり統計）
# 3. 前処理（欠損補完・変換）
# 4. 可視化で全体をつかむ（EDA）
# 5. 前処理後のデータを書き出し
# 6. 相関ヒートマップ

In [33]:
import os
os.makedirs("figs", exist_ok=True)
%cd "C:\Users\chino\pystudy\python-data-preprocessing"

# 1. データ読込
import pandas as pd
df = pd.read_csv("train.csv")

# 2. データの状態確認（欠損・型・ざっくり統計）
print("行数・列数:", df.shape)
print("\n欠損数（多い順）:"); print(df.isna().sum().sort_values(ascending=False))
print("\nデータ型:"); print(df.dtypes)
print("\n数値列の統計量:"); print(df.describe())
print("\nカテゴリ列のユニーク数:"); print(df.select_dtypes(exclude=["number"]).nunique())

C:\Users\chino\pystudy\python-data-preprocessing
行数・列数: (891, 12)

欠損数（多い順）:
Cabin          687
Age            177
Embarked         2
PassengerId      0
Name             0
Pclass           0
Survived         0
Sex              0
Parch            0
SibSp            0
Fare             0
Ticket           0
dtype: int64

データ型:
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

数値列の統計量:
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.0

In [29]:
# 3. 前処理（欠損補完・変換）

# Age：外れ値に強い中央値で埋める
df["Age"] = df["Age"].fillna(df["Age"].median())

# Embarked：欠損は2件だけ→最頻値で埋める
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

# Cabin：詳細は欠損だらけ→「記録があるか」を0/1化
df["HasCabin"] = df["Cabin"].notna().astype(int)

# ===== ② 文字を数値へ =====
# Sex：female=1, male=0（2値）
df["Sex_female"] = (df["Sex"] == "female").astype(int)

# Embarked：順序を持たせないため one-hot（S/C/Q をそれぞれ列に）
emb = pd.get_dummies(df["Embarked"], prefix="Embarked", dtype=int)
df = pd.concat([df, emb], axis=1)

# ===== ③ 使わない列を落とす（今回の練習では）=====
drop_cols = ["Name", "Ticket", "Cabin", "PassengerId", "Sex", "Embarked"]
df_clean = df.drop(columns=drop_cols)

# ===== ④ 保存（あとで学習に使える形）=====
df_clean.to_csv("cleaned_titanic.csv", index=False)
print("saved:", "cleaned_titanic.csv")

# 確認
print(df_clean.head())
print("shape:", df_clean.shape)

saved: cleaned_titanic.csv
   Survived  Pclass   Age  SibSp  Parch     Fare  HasCabin  Sex_female  \
0         0       3  22.0      1      0   7.2500         0           0   
1         1       1  38.0      1      0  71.2833         1           1   
2         1       3  26.0      0      0   7.9250         0           1   
3         1       1  35.0      1      0  53.1000         1           1   
4         0       3  35.0      0      0   8.0500         0           0   

   Embarked_C  Embarked_Q  Embarked_S  
0           0           0           1  
1           1           0           0  
2           0           0           1  
3           0           0           1  
4           0           0           1  
shape: (891, 11)


In [31]:
# 4. 可視化で全体をつかむ（EDA）
# 5. 前処理後のデータを書き出し

import os, matplotlib.pyplot as plt, seaborn as sns
os.makedirs("figs", exist_ok=True)

# 年齢ヒストグラム
plt.figure(figsize=(6,4))
plt.hist(df["Age"].dropna(), bins=30, edgecolor="black", color="skyblue")
plt.title("Age Distribution of Passengers")
plt.xlabel("Age (years)")
plt.ylabel("Number of passengers")
plt.tight_layout()
plt.savefig("figs/age_hist.png", dpi=150)
plt.close()

# 男女の人数（warning回避）
plt.figure(figsize=(4,4))
sns.countplot(data=df, x="Sex", hue="Sex", palette="Set2", legend=False)
plt.title("Passenger Count by Sex")
plt.xlabel("Sex")
plt.ylabel("Number of passengers")
plt.tight_layout()
plt.savefig("figs/sex_count.png", dpi=150)
plt.close()

print("Saved: figs/age_hist.png, figs/sex_count.png")

Saved: figs/age_hist.png, figs/sex_count.png


In [32]:


plt.figure(figsize=(6,5))
sns.heatmap(df_clean.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation between numeric features")
plt.tight_layout()
plt.savefig("figs/corr_numeric.png", dpi=150)
plt.close()

print("Saved: figs/corr_numeric.png")

Saved: figs/corr_numeric.png
